<font size="+3.8">Scanpy single-cell pre-processing</font>  
<font size="+1.5"></font>  

Aim: Preprocess mouse brain single-cell data from Ximerakis 2019 Nat Neuroscience

In [ ]:
from datetime import date
date.today().strftime('%d/%m/%Y')

In [ ]:
import os
os.getlogin()

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
import platform
import random
import seaborn as sns

In [ ]:
import utils

In [ ]:
os.environ['CONDA_DEFAULT_ENV'] # conda env

In [ ]:
platform.platform()

In [ ]:
sc.settings.verbosity = 3

In [ ]:
main_dir='/run/user/1000/gvfs/smb-share:server=138.245.4.35,share=bd-dichgans/SF' # Linux

In [ ]:
main_dir='\\\isdsynnas.srv.med.uni-muenchen.de\BD-Dichgans\SF' # Win

In [ ]:
dataset_name = "Ximerakis2019"
organism = "Mouse"

# Load data

Downloaded from: https://singlecell.broadinstitute.org/single_cell/study/SCP263/aging-mouse-brain 06/03/2023

In [ ]:
ad = sc.read_text(os.path.join(main_dir, "P06_vasc_scRNAseq", "Ximerakis2019", "expression_Aging_mouse_brain_portal_data_updated.txt")).T
ad

In [ ]:
ad.obs.head()

In [ ]:
ad.var_names

In [ ]:
# add metadata
meta = pd.read_csv(os.path.join(main_dir, "P06_vasc_scRNAseq", "Ximerakis2019", "meta_Aging_mouse_brain_portal_data.txt"), sep="\t", skiprows=[1])
meta.head()

In [ ]:
assert all(ad.obs.index == meta.NAME)
ad.obs = meta
ad

In [ ]:
pd.value_counts(ad.obs["cell_class"])

In [ ]:
pd.value_counts(ad.obs["cell_type"])

In [ ]:
ad.obs["age"] = ad.obs["all_cells_by_age"]
pd.value_counts(ad.obs["age"])

In [ ]:
pd.crosstab(ad.obs["cell_type"], ad.obs["cell_class"])

In [ ]:
# also see website

In [ ]:
new_names = {
    'Endothelial cells': {'EC'},
    'Oligos': {'OLG'},
    'Olfactory ensheathing glia': {'OEG'},
    'Astrocytes': {'ARP','ASC'},
    'Pericytes': {'PC'},
    'SMCs': {'VSMC'},
    'Neurons': {'NRP','NEUR_immature','NEUR_mature','NendC'},
    'OPCs': {'OPC'},
    'VLMCs': {'VLMC','ABC'},
    'Microglia/Macrophages': {'MG','MAC'},
    'Neural stem cells': {'NSC'},
    'Ependymal cells': {'EPC'},
    'Hypendymal cells': {'HypEPC','TNC'},
    'Choroid plexus epithelial cells':{'CPC'},
    'Immune_Other': {'MNC','DC','NEUT'},
    'Hemoglobin-expressing vascular cells':{'Hb_VC'},
}
result = {}
for key, value in new_names.items():
    for elem in value:
        result[(elem)] = key
new_cluster_names = dict(sorted(result.items()))
print(new_cluster_names)

In [ ]:
ad.obs['clusters'] = (
    ad.obs['cell_type']
    .map(new_cluster_names)
    .astype('category')
)

In [ ]:
pd.value_counts(ad.obs["clusters"])

In [ ]:
sc.pl.dotplot(ad, var_names=['Cldn5',"Aqp4","Mbp",'Foxj1','Pdgfrb','Vtn','Kcnj8','Rgs5','Atp13a5','Tagln','Acta2','Myh11'], groupby="clusters", swap_axes=False);

Last 3 cols are SMC markers

Check if data is normalized:

In [ ]:
ad.layers

In [ ]:
sns.histplot(ad.X.sum(1), kde=False)
plt.show()

In [ ]:
# show expression of 100 random genes (across all spots)
random_genes=random.sample(range(0, ad.X.shape[1]), 100)
adata_sub = ad[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
pl1=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3) # genes with 0 expression are excluded
pl1.set(xlim=(-0.5, 7),ylim=(0,0.007));
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(4,4)})
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 10));
sns.set_style("ticks")
plt.show()

In [ ]:
np.matrix(adata_sub.X)

Data seems to be normalized

In [ ]:
ad.layers["normalized"] = ad.X.copy() # save normalized + log-transformed (but unscaled) counts - retrieve via adata.X = adata.layers["normalized"]

In [ ]:
ad

In [ ]:
ad.layers

In [ ]:
sc.tl.pca(ad)
sc.pp.neighbors(ad)
sc.tl.umap(ad)
sc.tl.leiden(ad)

In [ ]:
sc.pl.umap(ad, color=['clusters'])

In [ ]:
sc.pl.umap(ad, color=['Foxf2'])

In [ ]:
adata=ad
del ad

# Focus on: Foxf2

In [ ]:
adata_backup = adata

In [ ]:
# drop aged samples
# adata = adata[adata.obs['age'].isin(['2-3mo'])]

In [ ]:
gene="Foxf2"

## Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
utils.summarize_gene_expression(adata = adata, gene = gene, groupby = "clusters", 
                                study_name = dataset_name, organism = organism,
                                export = True, output_dir = os.path.join(main_dir, "P06_Foxf2_per_celltype", "Foxf2_summarized")
                               )

# Focus on: Other genes

In [ ]:
target_genes = ["Foxo1", "Tek", "Nos3", "Htra1", "Egfl8", "Flt1", "Kdr", "Ptprb", "Nrp1", "Nrp2", "Efnb2", "Itgb1", "Itga6", "Angpt2", "Cdh5", "Cldn5", "Ocln", "Ctnnb1"]

In [ ]:
other_genes_results = {
    gene: utils.summarize_gene_expression(adata, gene, study_name = dataset_name, organism = organism, groupby = "clusters",
                                          output_dir=os.path.join(main_dir, "P06_Foxf2_per_celltype", "Other_genes_summarized"), export=True
                                         ) for gene in target_genes
}

In [ ]:
# some plots

In [ ]:
sc.pl.matrixplot(adata, [target_genes[0]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [target_genes[0]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [target_genes[1]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [target_genes[1]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

# Focus on: Aging

In [ ]:
adata = adata_backup

In [ ]:
pd.value_counts(adata.obs["age"])

## Foxf2 by age group

In [ ]:
gene=["Foxf2","Foxo1","Tek","Nos3"]

### All cell types

In [ ]:
sc.pl.dotplot(adata, gene, groupby='age', swap_axes=True, figsize=(4,2), standard_scale="var", layer="normalized", title="All cell types")

### ECs only

In [ ]:
# ECs only
adata_ecs = adata[adata.obs['clusters'].isin(['Endothelial cells'])]
adata_ecs

In [ ]:
pd.value_counts(adata_ecs.obs["age"])

In [ ]:
sc.pl.dotplot(adata_ecs, gene, groupby='age', swap_axes=True, figsize=(4,2), standard_scale="var", layer="normalized", title="ECs")

### PCs only

In [ ]:
adata_pcs = adata[adata.obs['clusters'].isin(['Pericytes'])]
pd.value_counts(adata_pcs.obs["age"])

In [ ]:
sc.pl.dotplot(adata_pcs, gene, groupby='age', swap_axes=True, figsize=(4,2), standard_scale="var", layer="normalized", title="PCs")

### SMCs only

In [ ]:
adata_pcs = adata[adata.obs['clusters'].isin(['SMCs'])]
pd.value_counts(adata_pcs.obs["age"])

In [ ]:
sc.pl.dotplot(adata_pcs, gene, groupby='age', swap_axes=True, figsize=(4,2), standard_scale="var", layer="normalized", title="SMCs")

### Neurons only

In [ ]:
adata_neur = adata[adata.obs['clusters'].isin(['Neurons'])]
pd.value_counts(adata_neur.obs["age"])

In [ ]:
sc.pl.dotplot(adata_neur, gene, groupby='age', swap_axes=True, figsize=(4,2), standard_scale="var", layer="normalized", title="Neurons")

### Astrocytes only

In [ ]:
adata_astro = adata[adata.obs['clusters'].isin(['Astrocytes'])]
pd.value_counts(adata_astro.obs["age"])

In [ ]:
sc.pl.dotplot(adata_astro, gene, groupby='age', swap_axes=True, figsize=(4,2), standard_scale="var", layer="normalized", title="Astrocytes")

### Oligos only

In [ ]:
adata_olig = adata[adata.obs['clusters'].isin(['Oligos'])]
pd.value_counts(adata_olig.obs["age"])

In [ ]:
sc.pl.dotplot(adata_olig, gene, groupby='age', swap_axes=True, figsize=(4,2), standard_scale="var", layer="normalized", title="Oligos")

### Microglia only

In [ ]:
adata_mg = adata[adata.obs['clusters'].isin(['Microglia/Macrophages'])]
pd.value_counts(adata_mg.obs["age"])

In [ ]:
sc.pl.dotplot(adata_mg, gene, groupby='age', swap_axes=True, figsize=(4,2), standard_scale="var", layer="normalized", title="Microglia/Macrophages")

## Correlate gene expression (Foxf2 and Foxo1)

### ECs

In [ ]:
gg = ["Foxf2","Foxo1"]

In [ ]:
adata

In [ ]:
adata_ecs

In [ ]:
adat = adata_ecs
subs = adat[:,gg].X
subs = pd.DataFrame(subs, columns = gg, index=adat.obs.index)
age = pd.DataFrame(adat.obs["age"])
subs = subs.join(age)
subs.head()

In [ ]:
g = sns.lmplot(x="Foxf2", y="Foxo1", data=subs, hue="age", fit_reg=False, scatter_kws={'alpha':0.6})
sns.regplot(x="Foxf2", y="Foxo1", data=subs, scatter=False, ax=g.axes[0, 0], color="grey")
g.figure.set_size_inches(7,6)
plt.show()

In [ ]:
import scipy as sp
r, p = sp.stats.pearsonr(subs['Foxf2'], subs['Foxo1'])
'Pearson R = {:.2f}, p = {:.2g}'.format(r, p)

In [ ]:
graph = sns.jointplot(data=subs, x="Foxf2", y="Foxo1", hue="age")
plt.show()

Exclude dropouts (cells with 0 expression of both genes)

In [ ]:
coex = (adata_ecs[:,'{}'.format("Foxf2")].X > 0) | (adata_ecs[:,'{}'.format("Foxo1")].X > 0)

In [ ]:
adata_ecs_above0 = adata_ecs[coex]
adata_ecs_above0

In [ ]:
pd.value_counts(adata_ecs_above0.obs["age"])

In [ ]:
sc.pl.dotplot(adata_ecs_above0, gene, groupby='age', swap_axes=True, figsize=(4,2), standard_scale="var", layer="normalized", title="ECs")

In [ ]:
adat = adata_ecs_above0
subs = adat[:,gg].X
subs = pd.DataFrame(subs, columns = gg, index=adat.obs.index)
age = pd.DataFrame(adat.obs["age"])
subs = subs.join(age)

In [ ]:
g = sns.lmplot(x="Foxf2", y="Foxo1", data=subs, hue="age", fit_reg=False, scatter_kws={'alpha':0.6})
sns.regplot(x="Foxf2", y="Foxo1", data=subs, scatter=False, ax=g.axes[0, 0], color="grey")
g.figure.set_size_inches(7,6)
plt.show()

In [ ]:
r, p = sp.stats.pearsonr(subs['Foxf2'], subs['Foxo1'])
'Pearson R = {:.2f}, p = {:.2g}'.format(r, p)

In [ ]:
graph = sns.jointplot(data=subs, x="Foxf2", y="Foxo1", hue="age")
plt.show()

### All cell types

In [ ]:
gg = ["Foxf2","Foxo1"]

In [ ]:
adata

In [ ]:
adat = adata
subs = adat[:,gg].X
subs = pd.DataFrame(subs, columns = gg, index=adat.obs.index)
age = pd.DataFrame(adat.obs["age"])
subs = subs.join(age)
subs.head()

In [ ]:
g = sns.lmplot(x="Foxf2", y="Foxo1", data=subs, hue="age", fit_reg=False, scatter_kws={'alpha':0.6})
sns.regplot(x="Foxf2", y="Foxo1", data=subs, scatter=False, ax=g.axes[0, 0], color="grey")
g.figure.set_size_inches(7,6)
plt.show()

In [ ]:
graph = sns.jointplot(data=subs, x="Foxf2", y="Foxo1", hue="age")
plt.show()

Exclude dropouts (cells with 0 expression of both genes)

In [ ]:
coex = (adat[:,'{}'.format("Foxf2")].X > 0) | (adat[:,'{}'.format("Foxo1")].X > 0)

In [ ]:
adata_above0 = adat[coex]
adata_above0

In [ ]:
pd.value_counts(adata_above0.obs["age"])

In [ ]:
sc.pl.dotplot(adata_above0, gene, groupby='age', swap_axes=True, figsize=(4,2), standard_scale="var", layer="normalized")

In [ ]:
adat = adata_above0
subs = adat[:,gg].X
subs = pd.DataFrame(subs, columns = gg, index=adat.obs.index)
age = pd.DataFrame(adat.obs["age"])
subs = subs.join(age)

In [ ]:
g = sns.lmplot(x="Foxf2", y="Foxo1", data=subs, hue="age", fit_reg=False, scatter_kws={'alpha':0.6})
sns.regplot(x="Foxf2", y="Foxo1", data=subs, scatter=False, ax=g.axes[0, 0], color="grey")
g.figure.set_size_inches(7,6)
plt.show()

In [ ]:
graph = sns.jointplot(data=subs, x="Foxf2", y="Foxo1", hue="age")
plt.show()

## Pseudobulk analysis (decoupler)

Because correlation on single cell level is biased by large number of dropouts

In [ ]:
import decoupler as dc

In [ ]:
adata

In [ ]:
adata.obs["sample"] = adata.obs['NAME'].str.extract('_([0-9]+)_?')
pd.value_counts(adata.obs["sample"])

In [ ]:
pd.crosstab(adata.obs["sample"], adata.obs["age"]).T

In [ ]:
padata = dc.get_pseudobulk(adata, sample_col='sample', groups_col=None, layer='normalized', min_prop=0.01, min_smpls=0)

In [ ]:
padata

### Correlate gene expression (Foxf2 and Foxo1)

#### All cell types

In [ ]:
gg = ["Foxf2","Foxo1"]

In [ ]:
padata

In [ ]:
adat = padata
subs = adat[:,gg].X
subs = pd.DataFrame(subs, columns = gg, index=adat.obs.index)
age = pd.DataFrame(adat.obs["age"])
subs = subs.join(age)
subs.head()

In [ ]:
g = sns.lmplot(x="Foxf2", y="Foxo1", data=subs, hue="age", fit_reg=False, scatter_kws={'alpha':0.6})
sns.regplot(x="Foxf2", y="Foxo1", data=subs, scatter=False, ax=g.axes[0, 0], color="grey")
g.figure.set_size_inches(7,6)
plt.show()

In [ ]:
import scipy as sp
r, p = sp.stats.pearsonr(subs['Foxf2'], subs['Foxo1'])
'Pearson R = {:.2f}, p = {:.2g}'.format(r, p)

In [ ]:
graph = sns.jointplot(data=subs, x="Foxf2", y="Foxo1", hue="age")
plt.show()

#### ECs

In [ ]:
padata = dc.get_pseudobulk(adata, sample_col='sample', groups_col='clusters', layer='normalized', min_prop=0.2, min_smpls=3)

In [ ]:
padata

In [ ]:
# ECs only
padata_ecs = padata[padata.obs['clusters'].isin(['Endothelial cells'])]
padata_ecs

In [ ]:
gg = ["Foxf2","Foxo1"]

In [ ]:
adat = padata_ecs
subs = adat[:,gg].X
subs = pd.DataFrame(subs, columns = gg, index=adat.obs.index)
age = pd.DataFrame(adat.obs["age"])
subs = subs.join(age)
subs.head()

In [ ]:
g = sns.lmplot(x="Foxf2", y="Foxo1", data=subs, hue="age", fit_reg=False, scatter_kws={'alpha':0.6})
sns.regplot(x="Foxf2", y="Foxo1", data=subs, scatter=False, ax=g.axes[0, 0], color="grey")
g.figure.set_size_inches(7,6)
plt.show()

In [ ]:
import scipy as sp
r, p = sp.stats.pearsonr(subs['Foxf2'], subs['Foxo1'])
'Pearson R = {:.2f}, p = {:.2g}'.format(r, p)

In [ ]:
graph = sns.jointplot(data=subs, x="Foxf2", y="Foxo1", hue="age")
plt.show()

## Denoising (MAGIC)

In [ ]:
import magic
import scprep

### Correlate gene expression (Foxf2 and Foxo1)

#### ECs

In [ ]:
gg = ["Foxf2","Foxo1"]

In [ ]:
adata

In [ ]:
adata_EC = adata[adata.obs.clusters == "Endothelial cells"]

In [ ]:
adata_EC

In [ ]:
#adata_EC = adata_EC[adata_EC.obs.age == "21-22mo"]

In [ ]:
adata_EC

In [ ]:
matrix = pd.DataFrame(adata_EC.X)
matrix.columns = adata_EC.var.index.tolist()

In [ ]:
matrix.head()

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=1500)

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata_EC.layers

##### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

##### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(matrix, genes=['Foxf2', 'Foxo1', 'Nos3'])

##### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

##### All ECs

In [ ]:
np.corrcoef(emt_magic[['Foxf2','Foxo1']], rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))

scprep.plot.scatter(x=matrix['Foxf2'], y=matrix['Foxo1'], c=matrix['Nos3'],  ax=ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')

scprep.plot.scatter(x=emt_magic['Foxf2'], y=emt_magic['Foxo1'], c=emt_magic['Nos3'], ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((0.2,0.2), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

##### 21-22mo EC only

In [ ]:
np.corrcoef(emt_magic[['Foxf2','Foxo1']], rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))

scprep.plot.scatter(x=matrix['Foxf2'], y=matrix['Foxo1'], c=matrix['Nos3'],  ax=ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')

scprep.plot.scatter(x=emt_magic['Foxf2'], y=emt_magic['Foxo1'], c=emt_magic['Nos3'], ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((0.2,0.2), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

##### 2-3mo ECs only

In [ ]:
np.corrcoef(emt_magic[['Foxf2','Foxo1']], rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))

scprep.plot.scatter(x=matrix['Foxf2'], y=matrix['Foxo1'], c=matrix['Nos3'],  ax=ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')

scprep.plot.scatter(x=emt_magic['Foxf2'], y=emt_magic['Foxo1'], c=emt_magic['Nos3'], ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((0.2,0.2), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

#### PCs

In [ ]:
adata_EC = adata[adata.obs.clusters == "Pericytes"]

In [ ]:
matrix = pd.DataFrame(adata_EC.X)
matrix.columns = adata_EC.var.index.tolist()

In [ ]:
matrix.head()

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=1500)

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata_EC.layers

##### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

##### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(matrix, genes=['Foxf2', 'Foxo1', 'Nos3'])

##### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[['Foxf2','Foxo1']], rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))

scprep.plot.scatter(x=matrix['Foxf2'], y=matrix['Foxo1'], c=matrix['Nos3'],  ax=ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')

scprep.plot.scatter(x=emt_magic['Foxf2'], y=emt_magic['Foxo1'], c=emt_magic['Nos3'], ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((0.2,0.2), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

#### SMCs

In [ ]:
adata_EC = adata[adata.obs.clusters == "SMCs"]

In [ ]:
matrix = pd.DataFrame(adata_EC.X)
matrix.columns = adata_EC.var.index.tolist()

In [ ]:
matrix.head()

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=1500)

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata_EC.layers

##### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

##### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(matrix, genes=['Foxf2', 'Foxo1', 'Nos3'])

##### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[['Foxf2','Foxo1']], rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))

scprep.plot.scatter(x=matrix['Foxf2'], y=matrix['Foxo1'], c=matrix['Nos3'],  ax=ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')

scprep.plot.scatter(x=emt_magic['Foxf2'], y=emt_magic['Foxo1'], c=emt_magic['Nos3'], ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((0.2,0.2), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

#### All cell types

In [ ]:
gg = ["Foxf2","Foxo1"]

In [ ]:
adata

In [ ]:
matrix = pd.DataFrame(adata.X)
matrix.columns = adata.var.index.tolist()

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=1500)

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata.layers

##### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

##### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(matrix, genes=['Foxf2', 'Foxo1', 'Nos3'])

##### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[['Foxf2','Foxo1']], rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))

scprep.plot.scatter(x=matrix['Foxf2'], y=matrix['Foxo1'], c=matrix['Nos3'],  ax=ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')

scprep.plot.scatter(x=emt_magic['Foxf2'], y=emt_magic['Foxo1'], c=emt_magic['Nos3'], ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((0,0), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

# Session Info

In [ ]:
sc.logging.print_versions()